### Read behavioral data file

[stimulus file format in afni](https://afni.nimh.nih.gov/afni/community/board/read.php?1,158062,158065#msg-158065)

In [12]:
import pandas as pd
import numpy as np
import re
debug = False
sid = 2
sub = f"sub-{sid:04d}"

In [2]:
sub

'sub-0002'

In [19]:
logfile = [f"/media/DATA2/GripForce/beh/{sub}_ses-01_task-GFORCE_run-L.csv",
           f"/media/DATA2/GripForce/beh/{sub}_ses-01_task-GFORCE_run-R.csv"]

if debug:
    runs = ['01','02']
else:
    if sid % 2 == 1:
        runs = ['01','02']
    else:
        runs = ['02','01']
    
   

for i in range(2):
    data = pd.read_csv(logfile[i])
    #print(data.columns)
    data2 = data.loc[:,
                     ['Target_Size','Ins_key.rt','TARGET.started',
                      'Ins_key.started', 'ISItime', 'ISI_4.stopped',
                      'trials.thisN']]
    target_starts = data.loc[:, "TARGET.started"].dropna().reset_index()
    data3 = data2.iloc[target_starts['index'], :]

    #corrected for the abnormal application of force
    data3 = data3.assign(Target_onsets = \
                         data3.loc[:,"TARGET.started"]- \
                         data.loc[0, 'Ins_key.rt'] - \
                         data.loc[0, 'Ins_key.started']- 2.4) 
    
    data3.loc[:,"Target_onsets"] = np.ceil(data3["Target_onsets"])
    
    #data3.head()
    # 40% disc
    data_s6 = data3.loc[data3['Target_Size']==0.6,:]

    with open(re.sub("-[LR].csv",f"-{runs[i]}-40%.1D",logfile[i]), 'w') as file:
        if runs[i] == '01':
            file.write(data_s6.Target_onsets.to_string(index=False).replace("\n",""))
            file.write('\n* *\n')
        else:
            file.write('* *\n')
            file.write(data_s6.Target_onsets.to_string(index=False).replace("\n",""))



    data_s9 = data3.loc[data3['Target_Size']==0.9,:]
    with open(re.sub("-[LR].csv",f"-{runs[i]}-60%.1D",logfile[i]), 'w') as file:
        if runs[i]=='01':
            file.write(data_s9.Target_onsets.to_string(index=False).replace("\n",""))
            file.write('\n* *\n')
        else:
            file.write('* *\n')            
            file.write(data_s9.Target_onsets.to_string(index=False).replace("\n",""))

            

    

In [24]:
print(re.sub("-[LR].csv",f"-{runs[i]}-60%.1D",logfile[i]), logfile[i])
print(re.sub("-[LR].csv",f"-{runs[i]}-40%.1D",logfile[i]), logfile[i])

/media/DATA2/GripForce/beh/sub-0002_ses-01_task-GFORCE_run-01-60%.1D /media/DATA2/GripForce/beh/sub-0002_ses-01_task-GFORCE_run-R.csv
/media/DATA2/GripForce/beh/sub-0002_ses-01_task-GFORCE_run-01-40%.1D /media/DATA2/GripForce/beh/sub-0002_ses-01_task-GFORCE_run-R.csv


## Notes

1. sub-0002 has a damaged sub-0002_ses-01_task-GFORCE_run-L.csv, using sub-0002_ses-01_task-GFORCE_run-L_1.csv to replace.